In [1]:
# Загружаем бибилиотеку pandas
import pandas as pd

In [2]:
# Словарь с номерами и названями нужных нам столбцов
col_names = {
    1: 'surgery?',
    2: 'Age', 
    4: 'rectal temperature', 
    5: 'pulse', 
    6: 'respiratory rate', 
    7: 'temperature of extremities', 
    11: 'pain', 
    23: 'outcome'
}

# Наименования статистических показателей, значения которых будем определять
stat_names = [
    'Min', # Минимум
    'Max', # Максимум
    'Range', # Размах
    'Mean', # Среднее арифметическое
    'Mode', # Мода
    'Median', # Медиана
    'Q1', # 1-я квартиль
    'Q3', # 3-я квартиль
    'IRQ', # Межквартильный размах
    'SD', # Стандартное отклонение
    'Var', # Дисерсия
    'Outliers', # Количество выбросов
    'Missings' # Количество пропущенных значений
]

# Рабочий датафрейм с информацией о лошадях
horse_data = pd.DataFrame()
# Копия рабочего датафрейма, в ней будем заполнять пропуски
horse_data_fixed = pd.DataFrame()
# Сюда будем записывать статистические показатели
horse_stat = pd.DataFrame(index=list(col_names.values()), columns = stat_names)

# Читаем файл со всеми данными о лошадях
# В файле нет заголовка, поэтому исользуем header=None
# Пропуски '?' заменяем на пустые значения
horse_data_full = pd.read_csv('horse_data.csv', sep=',', header=None, na_values = '?')

# Отбираем нужные нам столбцы и записываем в рабочий датафрейм
# Из нужного нам номера столбца вычитаем 1, так как питон считает нумерацию с 0
horse_data[list(col_names.values())] = horse_data_full.iloc[:,[i - 1 for i in list(col_names.keys())]]
# Копируем рабочий датафрейм
horse_data_fixed = horse_data.copy(deep=True)

In [3]:
# surgery?
#        1 = лошади сделали операцию
#        2 = лошадь лечили без операции

# Минимальное и максимальное значения, размах и среднее арифметическое не имеют смысла, 
# так как данные качественные номинальные

# Мода показывает, что большинству лошадей сделали операцию
horse_stat.loc['surgery?', 'Mode'] = horse_data['surgery?'].mode()[0]

# Медиана, 1-я квартиль, 3-я квартиль, межквартильный размах, стандартное отклонение, дисперсия не имеют смысла, 
# так как данные качественные номинальные

# Выбросы - это всё, что не совпадает с 1 и 2, при этом не является пустым значением
outliers_condition = (horse_data['surgery?'] != 1) & (horse_data['surgery?'] != 2) & (horse_data['surgery?'].notna())
# Количество выбросов
horse_stat.loc['surgery?', 'Outliers'] = outliers_condition.sum()
# Выбросы отсутствуют

# Количество  пропущенных значений
horse_stat.loc['surgery?', 'Missings'] = horse_data['surgery?'].isna().sum()

# Смотрим найденный пропуск
# horse_data[horse_data['surgery?'].isna()]
# Лошадь испытывает прерывистую сильную боль (pain = 4), конечности прохладные (temperature of extremities = 3),
# пульс и частота дыхания выше нормы (pulse = 48, respiratory rate = 20) - значит стотояние серьёзное.
# Температура в норме (rectal temperature = 38), но в совокупности с приведёнными выше показателями 
# это может указывать на состояние шока.
# Таким образом, можно предположить необходимость операции (surgery? = 1)
horse_data_fixed.loc[horse_data[horse_data['surgery?'].isna()].index[:], 'surgery?'] = 1


In [4]:
# Age
#        1 = взрослая лошадь
#        2 = молодая лошадь

# Минимальное и максимальное значения, размах и среднее арифметическое не имеют смысла, 
# так как данные качественные номинальные

# Мода показывает, что большинство лошадей взрослые
horse_stat.loc['Age', 'Mode'] = horse_data['Age'].mode()[0]

# Медиана, 1-я квартиль, 3-я квартиль, межквартильный размах, стандартное отклонение, дисперсия не имеют смысла, 
# так как данные качественные номинальные

# Выбросы - это всё, что не совпадает с 1 и 2, при этом не является пустым значением
outliers_condition = (horse_data['Age'] != 1) & (horse_data['Age'] != 2) & (horse_data['Age'].notna())
# Значения всех выбросов одинаковые (Age = 9), при этом нет ни одного значения Age = 2,
# поэтому считаем, что это ошибка заполнения
# Заполним количество выбросов (в нашем случае, ошибок)
horse_stat.loc['Age', 'Outliers'] = outliers_condition.sum()
# Исправим ошибки (9 -> 2)
horse_data_fixed.loc[horse_data[horse_data['Age'] == 9].index[:], 'Age'] = 2

# Пропуски отсутствуют
horse_stat.loc['Age', 'Missings'] = horse_data['Age'].isna().sum()

In [5]:
# rectal temperature
#           - линейная
#           - в градусах цельсия
#           - повышение может возникнуть из-за инфекции
#           - может снизиться, когда животное в состоянии шока
#           - норма 37,8
#           - этот параметр обычно изменяется по мере прогрессирования болезни,
#             например, сначала может быть в норме, затем повышается из-за поражений, 
#             и возвращается в нормальный диапазон, когда лошадь впадает в состояние шока

# Минимальное, максимальное значения и размах
# Показывают в каких пределах изменялась температура лошадей
horse_stat.loc['rectal temperature', 'Min'] = Min_ = horse_data['rectal temperature'].min()
horse_stat.loc['rectal temperature', 'Max'] = Max_ = horse_data['rectal temperature'].max()
horse_stat.loc['rectal temperature', 'Range'] = Max_ - Min_

# Среднее арифметическое
# Показывает среднюю температуру всех лошадей, полезность сомнительна
horse_stat.loc['rectal temperature', 'Mean'] = Mean_ = round(horse_data['rectal temperature'].mean(), 1)

# Мода не имеют смысла, так как данные количественные непрерывные

# Медиана, 1-я квартиль, 3-я квартиль, межквартильный размах
# Необходимы для определения выбросов
horse_stat.loc['rectal temperature', 'Median'] = horse_data['rectal temperature'].median()
horse_stat.loc['rectal temperature', 'Q1'] = Q1 = horse_data['rectal temperature'].quantile(0.25)
horse_stat.loc['rectal temperature', 'Q3'] = Q3 = horse_data['rectal temperature'].quantile(0.75)
horse_stat.loc['rectal temperature', 'IRQ'] = IRQ = Q3 - Q1

# Стандартное отклонение, дисперсия 
# Показывают степень разброса температуры лошадей
horse_stat.loc['rectal temperature', 'SD'] = round(horse_data['rectal temperature'].std(), 2)
horse_stat.loc['rectal temperature', 'Var'] = round(horse_data['rectal temperature'].var(), 2)

# Выбросы - это всё, что не попадает в границы [Q1 - 1,5 * IRQ; Q3 + 1,5 * IRQ], при этом не является пустым значением
outliers_condition = ((horse_data['rectal temperature'] <= (Q1 - 1.5 * IRQ)) | (horse_data['rectal temperature'] >= (Q3 + 1.5 * IRQ))) & (horse_data['rectal temperature'].notna())
# Количество выбросов
horse_stat.loc['rectal temperature', 'Outliers'] = outliers_condition.sum()
# Выбросов достаточно много, но их значения похожи на правду, оставляем как есть
# Так как выбросы выглядят правдоподобно, то в дальнейших расчётах
# для оценки центра распределения используем среднее арифметическое

# Смотрим найденные пропуски
# horse_data[horse_data['rectal temperature'].isna()]
# Количество  пропущенных значений
horse_stat.loc['rectal temperature', 'Missings'] = horse_data['rectal temperature'].isna().sum()

# Пропусков много, разбиваем их по группам и думаем как заполнить
# Предположим, что боль - это главный показатель тяжести болезни

# Группа пропусков pain = 1
# Предположим, что лошади, не страдающие от боли (pain = 1), в большинстве своём больны в лёгкой форме
# Средняя температура составляет 38,1, что незначительно выше нормального значения
# Отберём пропуски с условием pain = 1
missings_condition = (horse_data['rectal temperature'].isna()) & (horse_data['pain'] == 1)
# Пропуски для этой группы заменим средним значением темературы (pain = 1)
horse_data_fixed.loc[horse_data[missings_condition].index[:], 'rectal temperature'] = round(horse_data[horse_data['pain'] == 1]['rectal temperature'].mean(), 1)

# Группа пропусков pain = 2
# Предположим, что в эту группу входят лошади с состоянием средней тяжести, состояние шока отсутствует.
# Это означает, что по мере ухудшения состояния температура будет возрастать, 
# о чём будет сигнализировать частота пульса, а также температура конечностей.
#
# Для лошадей с нормальными и тёплыми конечностями (temperature of extremities = 1, 2) 
# средняя температура составляет 38,1
missings_condition = (horse_data['rectal temperature'].isna()) & (horse_data['pain'] == 2) & (horse_data['temperature of extremities'] <= 2)
horse_data_fixed.loc[horse_data[missings_condition].index[:], 'rectal temperature'] = round(horse_data[(horse_data['pain'] == 2) & (horse_data['temperature of extremities'] <= 2)]['rectal temperature'].mean(), 1)
# Для лошадей с прохладными и холодными конечностями (temperature of extremities = 3, 4) 
# и пульсом ниже 90 средняя температура составляет 38,6
missings_condition = (horse_data['rectal temperature'].isna()) & (horse_data['pain'] == 2) & (horse_data['temperature of extremities'] >= 3) & (horse_data['pulse'] < 90)
horse_data_fixed.loc[horse_data[missings_condition].index[:], 'rectal temperature'] = round(horse_data[(horse_data['pain'] == 2) & (horse_data['temperature of extremities'] >= 3) & (horse_data['pulse'] < 90)]['rectal temperature'].mean(), 1)
# Для лошадей с прохладными и холодными конечностями (temperature of extremities = 3, 4) 
# и пульсом выше 90 средняя температура составляет 39,0
missings_condition = (horse_data['rectal temperature'].isna()) & (horse_data['pain'] == 2) & (horse_data['temperature of extremities'] >= 3) & (horse_data['pulse'] >= 90)
horse_data_fixed.loc[horse_data[missings_condition].index[:], 'rectal temperature'] = round(horse_data[(horse_data['pain'] == 2) & (horse_data['temperature of extremities'] >= 3) & (horse_data['pulse'] >= 90)]['rectal temperature'].mean(), 1)

# Группа пропусков pain = 3, 4, 5
# Предположим, что в эту группу входят серьёзно больные лошади, встречается состояние шока.
# 
# Для состояния шока характерны прохладные или холодные конечности (temperature of extremities = 3, 4),
# высокий пульс (pulse > 90) и нормальная или пониженная температура
# Cредняя температура составляет 37,8
missings_condition = (horse_data['rectal temperature'].isna()) & (horse_data['pain'] >= 3) & (horse_data['temperature of extremities'] >= 3) & (horse_data['pulse'] >= 90)
horse_data_fixed.loc[horse_data[missings_condition].index[:], 'rectal temperature'] = round(horse_data[(horse_data['pain'] >= 3) & (horse_data['temperature of extremities'] >= 3) & (horse_data['pulse'] >= 90)]['rectal temperature'].mean(), 1)
# Предположим, что лошади с нормальными и тёплыми конечностями (temperature of extremities = 1, 2)
# и высокой частотой пульса (pulse > 90) не находятся в состоянии шока
# Cредняя температура составляет 38,4
missings_condition = (horse_data['rectal temperature'].isna()) & (horse_data['pain'] >= 3) & (horse_data['temperature of extremities'] <= 2) & (horse_data['pulse'] >= 90)
horse_data_fixed.loc[horse_data[missings_condition].index[:], 'rectal temperature'] = round(horse_data[(horse_data['pain'] >= 3) & (horse_data['temperature of extremities'] <= 2) & (horse_data['pulse'] >= 90)]['rectal temperature'].mean(), 1)
# Предположим, что у оставшихся лошадей с холодными конечностями (temperature of extremities = 4) повышенная температура
# Cредняя температура составляет 38,3
missings_condition = (horse_data['rectal temperature'].isna()) & (horse_data['pain'] >= 3) & (horse_data['temperature of extremities'] == 4) & (horse_data['pulse'] < 90)
horse_data_fixed.loc[horse_data[missings_condition].index[:], 'rectal temperature'] = round(horse_data[(horse_data['pain'] >= 3) & (horse_data['temperature of extremities'] == 4) & (horse_data['pulse'] < 90)]['rectal temperature'].mean(), 1)
# У остальных лошадей средняя температура составляет 38,0
missings_condition = (horse_data['rectal temperature'].isna()) & (horse_data['pain'] >= 3) & (horse_data['temperature of extremities'] <= 3) & (horse_data['pulse'] < 90)
horse_data_fixed.loc[horse_data[missings_condition].index[:], 'rectal temperature'] = round(horse_data[(horse_data['pain'] >= 3) & (horse_data['temperature of extremities'] <= 3) & (horse_data['pulse'] < 90)]['rectal temperature'].mean(), 1)

# Остались ещё пропуски (22 штуки) 
# При этом наблюдаются пропуски в других столбцах, поэтому нет однозначного способа заполнения
# Заполняем эти пропуски средним арифметическим для всех лошадей
horse_data_fixed[horse_data_fixed['rectal temperature'].isna()] = Mean_

In [6]:
# pulse
#           - линейный
#           - измеряется в ударах в минуту
#           - отражает состояние сердца, норма для взрослых 30-40
#           - у спортивных лошадей может составлять 20 - 25
#           - из-за боли и шока может повышаться

# Минимальное, максимальное значения и размах
horse_stat.loc['pulse', 'Min'] = Min_ = horse_data['pulse'].min()
horse_stat.loc['pulse', 'Max'] = Max_ = horse_data['pulse'].max()
horse_stat.loc['pulse', 'Range'] = Max_ - Min_

# Среднее арифметическое
horse_stat.loc['pulse', 'Mean'] = Mean_ = round(horse_data['pulse'].mean(), 1)

# Мода не имеют смысла, так как данные количественные непрерывные

# Медиана, 1-я квартиль, 3-я квартиль, межквартильный размах
horse_stat.loc['pulse', 'Median'] = horse_data['pulse'].median()
horse_stat.loc['pulse', 'Q1'] = Q1 = horse_data['pulse'].quantile(0.25)
horse_stat.loc['pulse', 'Q3'] = Q3 = horse_data['pulse'].quantile(0.75)
horse_stat.loc['pulse', 'IRQ'] = IRQ = Q3 - Q1

# Стандартное отклонение, дисперсия 
horse_stat.loc['pulse', 'SD'] = round(horse_data['pulse'].std(), 1)
horse_stat.loc['pulse', 'Var'] = round(horse_data['pulse'].var(), 1)

# Выбросы 
outliers_condition = ((horse_data['pulse'] <= (Q1 - 1.5 * IRQ)) | (horse_data['pulse'] >= (Q3 + 1.5 * IRQ))) & (horse_data['pulse'].notna())
# Количество выбросов
horse_stat.loc['pulse', 'Outliers'] = outliers_condition.sum()
# Выбросов немного, эти данные соответствуют молодым лошадям (у них пульс выше, чем у взрослых)
# Так как выбросы выглядят правдоподобно, то используем их в дальнейших расчётах

# Смотрим найденные пропуски
# horse_data[horse_data['pulse'].isna()]
# Количество  пропущенных значений
horse_stat.loc['pulse', 'Missings'] = horse_data['pulse'].isna().sum()

# Пропусков много, заполняем их используя найденные выше зависимости
#
# Для группы пропусков pain = 1
# Средний пульс составляет 50
missings_condition = (horse_data['pulse'].isna()) & (horse_data['pain'] == 1)
horse_data_fixed.loc[horse_data[missings_condition].index[:], 'pulse'] = round(horse_data[(horse_data['pain'] == 1)]['pulse'].mean(), 0)
# 
# Для группы пропусков pain = 2
# Средний пульс составляет 81
missings_condition = (horse_data['pulse'].isna()) & (horse_data['pain'] == 2)
horse_data_fixed.loc[horse_data[missings_condition].index[:], 'pulse'] = round(horse_data[(horse_data['pain'] == 2)]['pulse'].mean(), 0)
# 
# В группе пропусков pain = 3, 4, 5
# для лошадей с прохладными и холодными конечностями (temperature of extremities = 3, 4)
# Средний пульс составляет 82
missings_condition = (horse_data['pulse'].isna()) & (horse_data['pain'] >= 3) & (horse_data['temperature of extremities'] >= 3)
horse_data_fixed.loc[horse_data[missings_condition].index[:], 'pulse'] = round(horse_data[(horse_data['pain'] >= 3) & (horse_data['temperature of extremities'] >= 3)]['pulse'].mean(), 0)
# 
# В группе пропусков pain = 3, 4, 5
# для лошадей с нормальными и тёплыми конечностями (temperature of extremities = 1, 2)
# пропуски отсутствуют!

In [7]:
# respiratory rate
#           - линейный
#           - норма от 8 до 10
#           - полезность сомнительна из-за колебаний

# Минимальное, максимальное значения и размах
horse_stat.loc['respiratory rate', 'Min'] = Min_ = horse_data['respiratory rate'].min()
horse_stat.loc['respiratory rate', 'Max'] = Max_ = horse_data['respiratory rate'].max()
horse_stat.loc['respiratory rate', 'Range'] = Max_ - Min_

# Среднее арифметическое
horse_stat.loc['respiratory rate', 'Mean'] = Mean_ = round(horse_data['respiratory rate'].mean(), 1)

# Мода не имеют смысла, так как данные количественные непрерывные

# Медиана, 1-я квартиль, 3-я квартиль, межквартильный размах
horse_stat.loc['respiratory rate', 'Median'] = horse_data['respiratory rate'].median()
horse_stat.loc['respiratory rate', 'Q1'] = Q1 = horse_data['respiratory rate'].quantile(0.25)
horse_stat.loc['respiratory rate', 'Q3'] = Q3 = horse_data['respiratory rate'].quantile(0.75)
horse_stat.loc['respiratory rate', 'IRQ'] = IRQ = Q3 - Q1

# Стандартное отклонение, дисперсия 
horse_stat.loc['respiratory rate', 'SD'] = round(horse_data['respiratory rate'].std(), 1)
horse_stat.loc['respiratory rate', 'Var'] = round(horse_data['respiratory rate'].var(), 1)

# Выбросы 
outliers_condition = ((horse_data['respiratory rate'] <= (Q1 - 1.5 * IRQ)) | (horse_data['respiratory rate'] >= (Q3 + 1.5 * IRQ))) & (horse_data['respiratory rate'].notna())
# Количество выбросов
horse_stat.loc['respiratory rate', 'Outliers'] = outliers_condition.sum()
# Выбросов достаточно много.
# В описании сказано, что частота дыхания может меняться в широких пределах. У больных лошадей повышенная частота дыхания,
# это выглядят правдоподобно, поэтому выбросы используем в дальнейших расчётах.

# Смотрим найденные пропуски
# horse_data[horse_data['respiratory rate'].isna()]
# Количество  пропущенных значений
horse_stat.loc['respiratory rate', 'Missings'] = horse_data['respiratory rate'].isna().sum()
# 
# Пропусков много, заполняем их
#
# Для группы пропусков pain = 1
# Средня частота дыхания составляет 21
missings_condition = (horse_data['respiratory rate'].isna()) & (horse_data['pain'] == 1)
horse_data_fixed.loc[horse_data[missings_condition].index[:], 'respiratory rate'] = round(horse_data[(horse_data['pain'] == 1)]['respiratory rate'].mean(), 0)

# Для группы пропусков pain = 2
# наблюдается зависимость от температуры
# Для (rectal temperature < 38) cредня частота дыхания составляет 26
missings_condition = (horse_data['respiratory rate'].isna()) & (horse_data['pain'] == 2) & (horse_data['rectal temperature'] < 38)
horse_data_fixed.loc[horse_data[missings_condition].index[:], 'respiratory rate'] = round(horse_data[(horse_data['pain'] == 2) & (horse_data['rectal temperature'] < 38)]['respiratory rate'].mean(), 0)
#
# Для (rectal temperature > 38) cредня частота дыхания составляет 37
missings_condition = (horse_data['respiratory rate'].isna()) & (horse_data['pain'] == 2) & (horse_data['rectal temperature'] >= 38)
horse_data_fixed.loc[horse_data[missings_condition].index[:], 'respiratory rate'] = round(horse_data[(horse_data['pain'] == 2) & (horse_data['rectal temperature'] >= 38)]['respiratory rate'].mean(), 0)

# Для группы пропусков pain = 3, 4, 5
# Для (rectal temperature < 38) cредня частота дыхания составляет 26
missings_condition = (horse_data['respiratory rate'].isna()) & (horse_data['pain'] >= 3) & (horse_data['rectal temperature'] < 38)
horse_data_fixed.loc[horse_data[missings_condition].index[:], 'respiratory rate'] = round(horse_data[(horse_data['pain'] >= 3) & (horse_data['rectal temperature'] < 38)]['respiratory rate'].mean(), 0)
#
# Для (rectal temperature > 38) cредня частота дыхания составляет 31
missings_condition = (horse_data['respiratory rate'].isna()) & (horse_data['pain'] >= 3) & (horse_data['rectal temperature'] >= 38)
horse_data_fixed.loc[horse_data[missings_condition].index[:], 'respiratory rate'] = round(horse_data[(horse_data['pain'] >= 3) & (horse_data['rectal temperature'] >= 38)]['respiratory rate'].mean(), 0)

# Остались ещё пропуски (11 штуки) 
# Заполняем эти пропуски средним арифметическим для всех лошадей
horse_data_fixed[horse_data_fixed['respiratory rate'].isna()] = Mean_

In [8]:
# temperature of extremities
#           - признак периферического кровообращения
#           - 1 - нормальный
#           - 2 - тёплый
#           - 3 - прохладный
#           - 4 - холодный
#           - 1 - прохладный и холодный указывают на шок
#           - 2 - горячий связан температурой

# Минимальное и максимальное значения, размах и среднее арифметическое не имеют смысла, 
# так как данные качественные порядковые

# Мода показывает, что среди всех лошадей чаще всего встречаются с прохладными конечностями (temperature of extremities = 3)
horse_stat.loc['temperature of extremities', 'Mode'] = horse_data['temperature of extremities'].mode()[0]

# Медиана, 1-я квартиль, 3-я квартиль, межквартильный размах, стандартное отклонение, дисперсия не имеют смысла, 
# так как данные качественные порядковые

# Выбросы - это всё, что не совпадает с 1, 2, 3, 4, при этом не является пустым значением
outliers_condition = (horse_data['temperature of extremities'] != 1) & (horse_data['temperature of extremities'] != 2) & (horse_data['temperature of extremities'] != 3) & (horse_data['temperature of extremities'] != 4) & (horse_data['temperature of extremities'].notna())
# Количество выбросов
horse_stat.loc['temperature of extremities', 'Outliers'] = outliers_condition.sum()
# Выбросы отсутствуют

# Количество  пропущенных значений
horse_stat.loc['temperature of extremities', 'Missings'] = horse_data['temperature of extremities'].isna().sum()

# Заполняем пропуски
#
# Для группы пропусков pain = 1
# чаще всего встречаются лошади с нормальной температурой конечностей (temperature of extremities = 1)
missings_condition = (horse_data['temperature of extremities'].isna()) & (horse_data['pain'] == 1)
horse_data_fixed.loc[horse_data[missings_condition].index[:], 'temperature of extremities'] = horse_data[(horse_data['pain'] == 1)]['temperature of extremities'].mode()[0]
#
# Для группы пропусков pain = 2
# при (rectal temperature < 38) 
# чаще всего встречаются лошади с нормальной температурой конечностей (temperature of extremities = 1)
missings_condition = (horse_data['temperature of extremities'].isna()) & (horse_data['pain'] == 2) & (horse_data['rectal temperature'] < 38)
horse_data_fixed.loc[horse_data[missings_condition].index[:], 'temperature of extremities'] = horse_data[(horse_data['pain'] == 2) & (horse_data['rectal temperature'] < 38)]['temperature of extremities'].mode()[0]
#
# при (rectal temperature > 38) 
# чаще всего встречаются лошади с прохладными конечностями (temperature of extremities = 3)
missings_condition = (horse_data['temperature of extremities'].isna()) & (horse_data['pain'] == 2) & (horse_data['rectal temperature'] >= 38)
horse_data_fixed.loc[horse_data[missings_condition].index[:], 'temperature of extremities'] = horse_data[(horse_data['pain'] == 2) & (horse_data['rectal temperature'] >= 38)]['temperature of extremities'].mode()[0]
#
# Для группы пропусков pain = 3, 4, 5
# чаще всего встречаются лошади с прохладными конечностями (temperature of extremities = 3)
missings_condition = (horse_data['temperature of extremities'].isna()) & (horse_data['pain'] >= 3)
horse_data_fixed.loc[horse_data[missings_condition].index[:], 'temperature of extremities'] = horse_data[(horse_data['pain'] >= 3)]['temperature of extremities'].mode()[0]
#
# Остались ещё пропуски (22 штуки) 
# Заполняем эти пропуски значением моды для всех лошадей
horse_data_fixed[horse_data_fixed['temperature of extremities'].isna()] = horse_data['temperature of extremities'].mode()[0]

In [9]:
# pain
#           - субъективное мнение об уровне боли лошади
#           - 1 - бодрый, нет боли
#           - 2 - подавленный
#           - 3 - прерывистая лёгкая
#           - 4 - прерывистая сильная
#           - 5 - длительная сильная
#           - это не дискретная переменная!
#           - чем выше боль, там вероятнее необходимость хиругрического вмешательства
#           - лечение может снижать уровень боли

# Минимальное и максимальное значения, размах и среднее арифметическое не имеют смысла, 
# так как данные качественные порядковые

# Мода показывает, что среди всех лошадей чаще всего встречаются с прерывистой лёгкой болью (pain = 3)
horse_stat.loc['pain', 'Mode'] = horse_data['pain'].mode()[0]

# Медиана, 1-я квартиль, 3-я квартиль, межквартильный размах, стандартное отклонение, дисперсия не имеют смысла, 
# так как данные качественные порядковые

# Выбросы - это всё, что не совпадает с 1, 2, 3, 4, 5, при этом не является пустым значением
outliers_condition = (horse_data['pain'] != 1) & (horse_data['pain'] != 2) & (horse_data['pain'] != 3) & (horse_data['pain'] != 4) & (horse_data['pain'] != 5) & (horse_data['pain'].notna())
# Количество выбросов
horse_stat.loc['pain', 'Outliers'] = outliers_condition.sum()
# Выбросы отсутствуют

# Количество  пропущенных значений
horse_stat.loc['pain', 'Missings'] = horse_data['pain'].isna().sum()
# Заполняем пропуски модой
horse_data_fixed.loc[horse_data['pain'].isna().index[:], 'pain'] = horse_data['pain'].mode()[0]

In [10]:
# outcome
#           - что случилось в результате
#           - 1 - выжил
#           - 2 - умер
#           - 3 - усыплён

# Минимальное и максимальное значения, размах и среднее арифметическое не имеют смысла, 
# так как данные качественные номинальные

# Мода показывает, что лошади чаще всего выживали
horse_stat.loc['outcome', 'Mode'] = horse_data['outcome'].mode()[0]

# Медиана, 1-я квартиль, 3-я квартиль, межквартильный размах, стандартное отклонение, дисперсия не имеют смысла, 
# так как данные качественные номинальные

# Выбросы - это всё, что не совпадает с 1, 2, 3, при этом не является пустым значением
outliers_condition = (horse_data['outcome'] != 1) & (horse_data['outcome'] != 2) & (horse_data['outcome'] != 3) & (horse_data['outcome'].notna())
# Количество выбросов
horse_stat.loc['outcome', 'Outliers'] = outliers_condition.sum()
# Выбросы отсутствуют

# Количество  пропущенных значений
horse_stat.loc['outcome', 'Missings'] = horse_data['outcome'].isna().sum()

# Смотрим найденный пропуск
# horse_data[horse_data['surgery?'].isna()]
# Хочется надеяться, что лошадку вылечили
horse_data_fixed.loc[horse_data['outcome'].isna().index[:], 'outcome'] = 1


In [11]:
# Статпоказатели
horse_stat

,Min,Max,Range,Mean,Mode,Median,Q1,Q3,IRQ,SD,Var,Outliers,Missings
surgery?,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,0,1
Age,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,24,0
rectal temperature,35.4,40.8,5.4,38.2,NaN,38.2,37.8,38.5,0.7,0.73,0.54,14,60
pulse,30,184,154,71.9,NaN,64,48,88,40,28.6,819.7,5,24
respiratory rate,8,96,88,30.4,NaN,24.5,18.5,36,17.5,17.6,311.2,17,58
temperature of extremities,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,0,56
pain,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,0,55
outcome,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,0,1


In [12]:
# Датафрейм с заполненными пропусками
horse_data_fixed

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2.0,1.0,38.5,66.0,28.0,3.0,3.0,1.0
1,1.0,1.0,39.2,88.0,20.0,3.0,3.0,1.0
2,2.0,1.0,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,2.0,39.1,164.0,84.0,4.0,3.0,1.0
4,3.0,3.0,3.0,3.0,3.0,3.0,3.0,1.0
...,...,...,...,...,...,...,...,...
295,1.0,1.0,39.0,120.0,70.0,4.0,3.0,1.0
296,2.0,1.0,37.2,72.0,24.0,3.0,3.0,1.0
297,1.0,1.0,37.5,72.0,30.0,4.0,3.0,1.0
298,1.0,1.0,36.5,100.0,24.0,3.0,3.0,1.0
